In [29]:
import geopandas as gpd
import pandas as pd
from glob import glob
from plotly.offline import plot
from shapely.geometry import Point

import re

%matplotlib inline


In [164]:
# 행정동 정보 
files = glob("*.shp")

shp = pd.concat([gpd.read_file(file,encoding = 'cp949') for file in files]).reset_index(drop = True)

df = pd.read_csv("당신의 사는곳을 알려주세요 9572172efb494f81a1243b25bad749d8.csv")
df = df[df.columns[:3]].dropna()

In [168]:
df[df.시도명 == '양주시'] #.. 으으

,Name,시도명,시군구명
3,안성준,양주시,백석읍


In [171]:
df.loc[df.시도명 == '양주시','시도명'] = '경기도'
df.loc[df.시군구명 == '백석읍','시군구명'] = '양주시'

# 띄어쓰기 제거 

In [176]:
megas = df.시도명.apply(lambda x: x.replace(" ",""))

sggs= df.시군구명.apply(lambda x: x.replace(" ",''))

In [ ]:
# 중심점 계산 후 리스트로 저장 

In [ ]:
for mega,sgg in zip(megas,sggs):

In [227]:

shp[((shp.MEGA_NM.apply(lambda x: re.search(mega,x)).isna()==False)&(shp.SIG_KOR_NM.apply(lambda x: re.search(sgg,x)).isna()==False))].centroid.tolist()


In [180]:
centroid = [shp[((shp.MEGA_NM.apply(lambda x: re.search(mega,x)).isna()==False)&(shp.SIG_KOR_NM.apply(lambda x: re.search(sgg,x)).isna()==False))].centroid.tolist()[0] for mega,sgg in zip(megas,sggs)]

df['centroid'] = centroid

In [188]:
df

,Name,시도명,시군구명,centroid
1,이종원,서울시,양천구,POINT (943049.1483472434 1947471.644039042)
2,이정훈,서울시,강서구,POINT (940191.387102065 1951535.447214201)
3,안성준,경기도,양주시,POINT (977391.3276712752 1962588.959609232)
4,박범수,서울시,동대문구,POINT (960695.9357565708 1953711.983295836)
5,최규진,서울시,서대문구,POINT (950470.0186996566 1953303.946911824)
6,김지선,서울시,중구,POINT (955484.1952980496 1951318.203971829)
7,김준서,인천시,부평구,POINT (931159.0238156538 1944440.436989765)
8,박광해,서울시,관악구,POINT (950951.7866165609 1941051.326338609)
9,이채욱,서울시,영등포구,POINT (947880.1097111189 1947164.667821349)
10,양건필,서울시,강남구,POINT (961370.4031234696 1944243.66399422)


In [184]:
df.to_csv("당신이 사는곳을.csv")

In [190]:
import plotly.express as px


In [193]:
from shapely.wkt import loads

In [196]:
df.centroid = df.centroid.apply(lambda x: loads(str(x)))

In [203]:
gdf = gpd.GeoDataFrame(df,geometry = centroid,crs = 5179)

In [211]:
ctdos = gdf[['시도명','시군구명']].drop_duplicates().시도명
ctys = gdf[['시도명','시군구명']].drop_duplicates().시군구명

In [219]:
for ctdo,cty in zip(ctdos,ctys):
    n = len(gdf[(gdf.시도명 ==ctdo)&(gdf.시군구명 == cty)])
    gdf[(gdf.시도명 ==ctdo)&(gdf.시군구명 == cty)].구분 = n

In [229]:
gdf['row_num'] = gdf.groupby(['시도명','시군구명']).cumcount() + 1

In [231]:
gdf.geometry.x + 50

1     943099.148347
2     940241.387102
3     977441.327671
4     960745.935757
5     950520.018700
6     955534.195298
7     931209.023816
8     951001.786617
9     947930.109711
10    961420.403123
11    945241.295145
12    954098.023714
13    948777.213309
14    951579.628126
15    974066.230835
16    950520.018700
17    931975.208986
18    925537.829393
19    940241.387102
dtype: float64

In [235]:
gdf['x'] = gdf.geometry.x
gdf['y'] = gdf.geometry.y

In [241]:
gdf.loc[gdf.row_num == 2,'x'] = gdf.loc[gdf.row_num == 2,'x'] + 50

In [246]:
gdf2 = gpd.GeoDataFrame(gdf,geometry = [Point(x,y) for x,y in zip(gdf.x,gdf.y)],crs = 5179)

In [247]:
gdf2 = gdf2.to_crs(4326)

In [249]:
gdf2['lon'] = gdf2.geometry.x
gdf2['lat'] = gdf2.geometry.y

In [250]:
gdf2

,Name,시도명,시군구명,centroid,geometry,row_num,x,y,lon,lat
1,이종원,서울시,양천구,POINT (943049.1483472434 1947471.644039042),POINT (126.85548 37.52479),1,943049.148347,1.947472e+06,126.855478,37.524789
2,이정훈,서울시,강서구,POINT (940191.387102065 1951535.447214201),POINT (126.82281 37.56124),1,940191.387102,1.951535e+06,126.822807,37.561235
3,안성준,경기도,양주시,POINT (977391.3276712752 1962588.959609232),POINT (127.24366 37.66253),1,977391.327671,1.962589e+06,127.243661,37.662530
4,박범수,서울시,동대문구,POINT (960695.9357565708 1953711.983295836),POINT (127.05485 37.58196),1,960695.935757,1.953712e+06,127.054848,37.581957
5,최규진,서울시,서대문구,POINT (950470.0186996566 1953303.946911824),POINT (126.93906 37.57779),1,950470.018700,1.953304e+06,126.939063,37.577785
6,김지선,서울시,중구,POINT (955484.1952980496 1951318.203971829),POINT (126.99597 37.56014),1,955484.195298,1.951318e+06,126.995968,37.560144
7,김준서,인천시,부평구,POINT (931159.0238156538 1944440.436989765),POINT (126.72121 37.49666),1,931159.023816,1.944440e+06,126.721209,37.496659
8,박광해,서울시,관악구,POINT (950951.7866165609 1941051.326338609),POINT (126.94534 37.46738),1,950951.786617,1.941051e+06,126.945337,37.467376
9,이채욱,서울시,영등포구,POINT (947880.1097111189 1947164.667821349),POINT (126.91017 37.52231),1,947880.109711,1.947165e+06,126.910169,37.522308
10,양건필,서울시,강남구,POINT (961370.4031234696 1944243.66399422),POINT (127.06299 37.49664),1,961370.403123,1.944244e+06,127.062985,37.496644


In [253]:
fig = px.scatter_mapbox(gdf2,lat = 'lat',lon = 'lon',hover_name = 'Name',)

In [263]:
fig.update_layout(mapbox_style = 'white-bg')

In [264]:
plot(fig, filename='notion_update2.html')

'notion_update2.html'